In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/PulkitChadha125/DataScienceProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="PulkitChadha125"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e89ec8d96f3d0d5e3c55540c3e74eff5e14d8b41"


In [2]:
%pwd

'd:\\Desktop\\MLOPS\\DataScienceProject\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Desktop\\MLOPS\\DataScienceProject'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass()
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
import importlib
import src.datascienceproject.constants
importlib.reload(src.datascienceproject.constants)
from src.datascienceproject.constants import *
from src.datascienceproject.utils.common import read_yaml,create_directories,save_json

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH,
                schema_filepath=SCHEMA_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])


        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/PulkitChadha125/DataScienceProject.mlflow")

        return model_evaluation_config


In [8]:
import os
import pandas as pd
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [11]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config

    def eval_metrics(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        return rmse,mae,r2
    
    
    def log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities=model.predict(test_x)
            (rmse,mae,r2)=self.eval_metrics(test_y,predicted_qualities)

            # save metric as local
            scores={
                "rmse":rmse,
                "mae":mae,
                "r2":r2
            }
            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model,
                                         "model",
                                         registered_model_name="ElasticModel")

            else:
                mlflow.sklearn.log_model(model,"model")
                
            
            
            
                

In [12]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e


[2025-10-17 23:03:37,895]:INFO:common:yaml file: config\config.yaml loaded successfully
[2025-10-17 23:03:37,897]:INFO:common:yaml file: params.yaml loaded successfully
[2025-10-17 23:03:37,898]:INFO:common:yaml file: schema.yaml loaded successfully
[2025-10-17 23:03:37,898]:INFO:common:created directory at: artifacts
[2025-10-17 23:03:37,898]:INFO:common:created directory at: artifacts/model_evaluation
[2025-10-17 23:03:38,297]:INFO:common:json file: artifacts\model_evaluation\metrics.json saved successfully


2025/10/17 23:03:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticModel'.
2025/10/17 23:03:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticModel, version 1
Created version '1' of model 'ElasticModel'.
2025/10/17 23:03:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run righteous-mare-584 at: https://dagshub.com/PulkitChadha125/DataScienceProject.mlflow/#/experiments/0/runs/d09678ce764841ea9392e88c80934c23.
2025/10/17 23:03:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/PulkitChadha125/DataScienceProject.mlflow/#/experiments/0.
